In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn

In [7]:
train = pd.read_csv('train.csv')
train = train[train.columns[::-1]]

test = pd.read_csv('test.csv')
test = test[test.columns[::-1]]

mean1 = train['mean1']
mean2 = train['mean2']
train.drop(['mean1', 'mean2'], axis=1, inplace=True)  
train

,cov,var2,var1,r2_t-10,r1_t-10,r2_t-9,r1_t-9,r2_t-8,r1_t-8,r2_t-7,...,r2_t-5,r1_t-5,r2_t-4,r1_t-4,r2_t-3,r1_t-3,r2_t-2,r1_t-2,r2_t-1,r1_t-1
0,0.000097,0.000059,0.000331,0.002647,0.014603,0.000705,0.006397,0.004221,-0.019597,0.003328,...,-0.009326,-0.033898,0.008446,0.013583,0.002705,-0.015634,-0.011224,-0.029495,0.012496,0.018703
1,0.000111,0.000093,0.000296,0.000705,0.006397,0.004221,-0.019597,0.003328,0.002161,0.001396,...,0.008446,0.013583,0.002705,-0.015634,-0.011224,-0.029495,0.012496,0.018703,-0.010169,-0.004016
2,0.000153,0.000131,0.000327,0.004221,-0.019597,0.003328,0.002161,0.001396,-0.014016,-0.009326,...,0.002705,-0.015634,-0.011224,-0.029495,0.012496,0.018703,-0.010169,-0.004016,-0.019229,-0.017857
3,0.000184,0.000132,0.000395,0.003328,0.002161,0.001396,-0.014016,-0.009326,-0.033898,0.008446,...,-0.011224,-0.029495,0.012496,0.018703,-0.010169,-0.004016,-0.019229,-0.017857,-0.022292,-0.034604
4,0.000175,0.000127,0.000424,0.001396,-0.014016,-0.009326,-0.033898,0.008446,0.013583,0.002705,...,0.012496,0.018703,-0.010169,-0.004016,-0.019229,-0.017857,-0.022292,-0.034604,0.005128,0.017011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,0.000073,0.000048,0.000316,0.004299,0.019530,0.003890,-0.018503,0.004567,0.022090,0.013790,...,-0.001647,0.011169,0.005625,0.024279,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270
3507,0.000072,0.000048,0.000295,0.003890,-0.018503,0.004567,0.022090,0.013790,0.009044,0.003209,...,0.005625,0.024279,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266
3508,0.000074,0.000049,0.000242,0.004567,0.022090,0.013790,0.009044,0.003209,0.005448,-0.001647,...,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266,0.004223,0.009195
3509,0.000072,0.000049,0.000212,0.013790,0.009044,0.003209,0.005448,-0.001647,0.011169,0.005625,...,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266,0.004223,0.009195,0.001808,0.002800


In [8]:
X = train.drop(["var1", "var2", "cov"], axis=1, inplace=False)
y = train[["var1", "var2", "cov"]].values  

In [ ]:
""" scaler = StandardScaler()
X = scaler.fit_transform(X)  """

SyntaxError: EOF while scanning triple-quoted string literal (4062223562.py, line 2)

In [ ]:
""" X = X.reshape((X.shape[0], 10, 2))
X.shape """

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
X[0]

array([[ 9.71331189e-05,  5.87031255e-05],
       [ 3.31260439e-04,  2.64707533e-03],
       [ 1.46025071e-02,  7.04514062e-04],
       [ 6.39663532e-03,  4.22073950e-03],
       [-1.95974338e-02,  3.32819877e-03],
       [ 2.16117360e-03,  1.39623525e-03],
       [-1.40162833e-02, -9.32621544e-03],
       [-3.38984894e-02,  8.44625997e-03],
       [ 1.35827374e-02,  2.70453211e-03],
       [-1.56340753e-02, -1.12243668e-02]])